In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dropout, Dense
from tensorflow.keras.utils import to_categorical

# Read data set
We have a data set containing song lyrics. We display the first 50 chars of such a dataset.

In [3]:
data = open('Zayn_Lyrics.txt').read()

In [4]:
data[:50]

"Now I'm on the edge\nI can't find my way\nIt's insid"

# Generate corpus
We work with lower case data. Each line of the song lyric is an entry for the corpus. We save it into a set to avoid duplicates.

In [5]:
# Splitting the string into sentences, while converting whole data into lowercase.
corpus = data.lower().split("\n")
# To make sure no sentence appears twice in our corpus, we use set. Otherwise, it will make the model biased.
corpus = list(set(corpus))
corpus

['',
 "can't find a way through",
 'light and dark',
 "i'd love to wake up next to you",
 "she likes when i'm messy",
 "i'm getting caught up in the feelings that they bring",
 "and it's all mine",
 'not like this',
 "this heart won't be at peace",
 "let's get naked and explore",
 "nobody but you, 'body but me, 'body but us",
 "because i just can't face it",
 'prisoners',
 "so pardon if i don't speak",
 'to kiss her',
 'i know sometimes i hide it',
 "'cause i'm a problem with problems",
 "don't know how many times",
 'maybe keep it on the low',
 "i'll take all the right wrongs",
 "i won't point any fingers",
 "and it ain't about who's winning or losing",
 "don't stick that knife in my body",
 "i'm living on a wire",
 "i don't care where we go",
 'and in time you see the truth',
 "i've had to talk you through",
 "because my brain can't equate it",
 'time heals pain and promotes self-soothing',
 "then we're free, it's a thriller",
 'four letters is never the question',
 "let's break down

We then separete each work and we tokenize the corpus using the Tokenizer library from keras. We also save a variable containing the total number of words.

In [6]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1
tokenizer.word_index

{'i': 1,
 'the': 2,
 'you': 3,
 'it': 4,
 'to': 5,
 'and': 6,
 'a': 7,
 'in': 8,
 "don't": 9,
 'all': 10,
 'my': 11,
 'me': 12,
 "i'm": 13,
 "it's": 14,
 'this': 15,
 'that': 16,
 'take': 17,
 "can't": 18,
 'she': 19,
 'like': 20,
 'on': 21,
 'off': 22,
 'just': 23,
 'so': 24,
 'right': 25,
 'of': 26,
 'but': 27,
 'for': 28,
 'what': 29,
 "won't": 30,
 'your': 31,
 "you're": 32,
 'know': 33,
 'time': 34,
 'when': 35,
 'be': 36,
 "'cause": 37,
 'say': 38,
 'wanna': 39,
 'way': 40,
 'love': 41,
 'see': 42,
 'is': 43,
 'no': 44,
 'up': 45,
 'about': 46,
 'things': 47,
 'got': 48,
 'her': 49,
 'baby': 50,
 'with': 51,
 "i'll": 52,
 'out': 53,
 'zone': 54,
 "she's": 55,
 'give': 56,
 'now': 57,
 'good': 58,
 'light': 59,
 'not': 60,
 'at': 61,
 'go': 62,
 'had': 63,
 'never': 64,
 'try': 65,
 "you've": 66,
 'paradise': 67,
 'war': 68,
 'do': 69,
 'place': 70,
 'hold': 71,
 'need': 72,
 'seeing': 73,
 'want': 74,
 'through': 75,
 'heart': 76,
 'get': 77,
 'because': 78,
 "ain't": 79,
 'or': 

We then create what is going to be our input sequences. We take the first line of the corpus _"climb on board"_ and we generate two new lines from it: _"climb on"_ and _"climb on board"_. These two line are represented using their token. We repeate this procedure for all input lines.

In [7]:
# create input sequences using list of tokens
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)
print(tokenizer.word_index["making"])
print(tokenizer.word_index["time"])
print(tokenizer.word_index["go"])
input_sequences

171
34
62


[[18, 97],
 [18, 97, 7],
 [18, 97, 7, 40],
 [18, 97, 7, 40, 75],
 [59, 6],
 [59, 6, 136],
 [98, 41],
 [98, 41, 5],
 [98, 41, 5, 205],
 [98, 41, 5, 205, 45],
 [98, 41, 5, 205, 45, 206],
 [98, 41, 5, 205, 45, 206, 5],
 [98, 41, 5, 205, 45, 206, 5, 3],
 [19, 137],
 [19, 137, 35],
 [19, 137, 35, 13],
 [19, 137, 35, 13, 207],
 [13, 208],
 [13, 208, 99],
 [13, 208, 99, 45],
 [13, 208, 99, 45, 8],
 [13, 208, 99, 45, 8, 2],
 [13, 208, 99, 45, 8, 2, 209],
 [13, 208, 99, 45, 8, 2, 209, 16],
 [13, 208, 99, 45, 8, 2, 209, 16, 100],
 [13, 208, 99, 45, 8, 2, 209, 16, 100, 210],
 [6, 14],
 [6, 14, 10],
 [6, 14, 10, 101],
 [60, 20],
 [60, 20, 15],
 [15, 76],
 [15, 76, 30],
 [15, 76, 30, 36],
 [15, 76, 30, 36, 61],
 [15, 76, 30, 36, 61, 138],
 [102, 77],
 [102, 77, 211],
 [102, 77, 211, 6],
 [102, 77, 211, 6, 212],
 [213, 27],
 [213, 27, 3],
 [213, 27, 3, 139],
 [213, 27, 3, 139, 27],
 [213, 27, 3, 139, 27, 12],
 [213, 27, 3, 139, 27, 12, 139],
 [213, 27, 3, 139, 27, 12, 139, 27],
 [213, 27, 3, 139, 27

In order to have arrays of same length we add some padding to the left to all the rows.

In [8]:
# pad sequences
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen = max_sequence_len, padding='pre'))
input_sequences

array([[  0,   0,   0, ...,   0,  18,  97],
       [  0,   0,   0, ...,  18,  97,   7],
       [  0,   0,   0, ...,  97,   7,  40],
       ...,
       [  0,   0,   0, ..., 511,  46,  29],
       [  0,   0,   0, ...,  46,  29,   1],
       [  0,   0,   0, ...,  29,   1,  72]], dtype=int32)

We separate what is going to be our inputs from our target values. For this purpose we consider, for instance, "climb" to be the input and "on" to be the target value, "climb on" to be the input next and "board" to be the target value. We use predictors as input and label as our target value.

In [9]:
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
print(predictors[:10])
print(label[:10])

[[  0   0   0   0   0   0   0   0   0   0   0   0  18]
 [  0   0   0   0   0   0   0   0   0   0   0  18  97]
 [  0   0   0   0   0   0   0   0   0   0  18  97   7]
 [  0   0   0   0   0   0   0   0   0  18  97   7  40]
 [  0   0   0   0   0   0   0   0   0   0   0   0  59]
 [  0   0   0   0   0   0   0   0   0   0   0  59   6]
 [  0   0   0   0   0   0   0   0   0   0   0   0  98]
 [  0   0   0   0   0   0   0   0   0   0   0  98  41]
 [  0   0   0   0   0   0   0   0   0   0  98  41   5]
 [  0   0   0   0   0   0   0   0   0  98  41   5 205]]
[ 97   7  40  75   6 136  41   5 205  45]


We train the model using word embeddings, lstm and a normal NN at the end. 

In [10]:
model = Sequential()
model.add(Embedding(total_words, 50, input_length=max_sequence_len-1))
# Add an LSTM Layer
model.add(Bidirectional(LSTM(150, return_sequences=True)))  
# A dropout layer for regularisation
model.add(Dropout(0.2))
# Add another LSTM Layer
model.add(LSTM(100)) 
model.add(Dense(total_words/2, activation='relu'))  
# In the last layer, the shape should be equal to the total number of words present in our corpus
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])  #(# Pick a loss function and an optimizer)
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 13, 50)            25600     
_________________________________________________________________
bidirectional (Bidirectional (None, 13, 300)           241200    
_________________________________________________________________
dropout (Dropout)            (None, 13, 300)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               160400    
_________________________________________________________________
dense (Dense)                (None, 256)               25856     
_________________________________________________________________
dense_1 (Dense)      

In [11]:
history = model.fit(predictors, label, epochs= 100, verbose=1)

Epoch 1/100
1496/1496 [==============================] - 5s 3ms/sample - loss: 5.8619 - acc: 0.0361
Epoch 2/100
1496/1496 [==============================] - 2s 2ms/sample - loss: 5.4937 - acc: 0.0394
Epoch 3/100
1496/1496 [==============================] - 3s 2ms/sample - loss: 5.4137 - acc: 0.0368
Epoch 4/100
1496/1496 [==============================] - 3s 2ms/sample - loss: 5.2695 - acc: 0.0455
Epoch 5/100
1496/1496 [==============================] - 2s 2ms/sample - loss: 5.1469 - acc: 0.0475
Epoch 6/100
1496/1496 [==============================] - 2s 2ms/sample - loss: 5.0569 - acc: 0.0461
Epoch 7/100
1496/1496 [==============================] - 2s 2ms/sample - loss: 4.9765 - acc: 0.0361
Epoch 8/100
1496/1496 [==============================] - 2s 1ms/sample - loss: 4.8994 - acc: 0.0515
Epoch 9/100
1496/1496 [==============================] - 2s 1ms/sample - loss: 4.8485 - acc: 0.0595
Epoch 10/100
1496/1496 [==============================] - 2s 2ms/sample - loss: 4.7766 - acc: 0.0475

# Predict some song lyrics using the trained model.

In [12]:
def make_lyrics(seed_text, next_words):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list],
                     maxlen=max_sequence_len-1,padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    print(seed_text)

In [13]:
make_lyrics("i'm watching", 10)

i'm watching tryna fuckin' scream but the words won't come out search
